[Professor GitHub Repository](https://github.com/bardhprenkaj/ML_labs/blob/main/resources/project/project_description.pdf)

[Project GitHub Repository](https://github.com/Naxetee/ML_FinalProject)

# Python Libraries


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt


# Task 1

During this first task we will apply some **pre-processing** operations to our raw data. So we will directly modify the data in *data/dataset/*, so that the data in *data/dataset_fixed/* will always remain unmodified.

## Reading the Datasets

At first, let's read the **.csv** files using a *pandas* function called **read_csv** whose parameters are:
- *filepath*: the path of the .csv files.
- *header*: will be always 0 to skip the columns' name row.
- *names*: gives a name to each column of the table.


In [ ]:
data_path = "../../data/dataset/sample"

ap = pd.read_csv(data_path+'/anagraficapazientiattivi.csv', header=0 ,names=['idcentro','idana','sesso','annodiagnosidiabete','tipodiabete','scolarita','statocivile','professione','origine','annonascita','annoprimoaccesso','annodecesso'])
diag = pd.read_csv(data_path+'/diagnosi.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
elp  = pd.read_csv(data_path+'/esamilaboratorioparametri.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
ei = pd.read_csv(data_path+'/esamistrumentali.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pdf = pd.read_csv(data_path+'/prescrizionidiabetefarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceatc','quantita','idpasto','descrizionefarmaco'])
pdnf = pd.read_csv(data_path+'/prescrizionidiabetenonfarmaci.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])
pnd = pd.read_csv(data_path+'/prescrizioninondiabete.csv', header=0 ,names=['idcentro','idana','data','codiceamd','valore'])

In [ ]:
# Display example observations
print(ap.shape)
ap.head()

In [ ]:
print(diag.shape)
diag.head()

In [ ]:
print(elp.shape)
elp.head()

In [ ]:
print(ei.shape)
ei.head()

In [ ]:
print(pdf.shape)
pdf.head()

In [ ]:
print(pdnf.shape)
pdnf.head()

In [ ]:
print(pnd.shape)
pnd.head()

## Exercise 1

**Select Events of Interest -**
We want only patients with at leats one cardiovascular event in their trajectories. These events have the folloging codes:
- **AMD047**: Myocardial infarction
- **AMD048**: Coronary angioplasty
- **AMD049**: Coronary bypass
- **AMD071**: Ictus
- **AMD081**: Lower limb angioplasty
- **AMD082**: Peripheral By-pass Lower Limbs
- **AMD208**: Revascularization of intracranial and neck vessels
- **AMD303**: Ischemic stroke

Now let's try to filter all the tables containing the parameter **codiceamd** so as to select just the rows describing one cardio-vascular event.


In [ ]:
codes = ['AMD047', 'AMD048', 'AMD049', 'AMD071', 'AMD081', 'AMD082', 'AMD208', 'AMD303']

def selectEvents(df, events_codes) -> pd.DataFrame:
    return df[df.codiceamd.isin(events_codes)]

## pnd_cardEvents = selectEvents(pnd,codes)

## pdnf_cardEvents = selectEvents(pdnf,codes)

## ei_cardEvents = selectEvents(ei,codes)

## elp_cardEvents = selectEvents(elp,codes)

diag_cardEvents = selectEvents(diag,codes)
print("BEFORE pre-processing: ", diag.shape)
print("AFTER  pre-processing: ", diag_cardEvents.shape)
diag_cardEvents.head()


We realise that the only table with these cardio-vascular events is **diagnosi**, so we create a new DataFrame with all the rows that has one of these events, and we call it: **diag_cardEvents**. And from this table, we extract the id's of the patients that have suffered at least one cardio-vascular event during his life:

In [ ]:
patientsOfInterest = diag_cardEvents.filter(['idcentro','idana']).drop_duplicates()
print("Number of patients with at least one cardio-vascular event: ", patientsOfInterest.shape[0], "/", ap.shape[0] , "patients.")

Finally, we have to delete from the rest of the tables all the information which is not associated to the remaining patients:

In [ ]:
ap_ex1 = pd.merge(ap, patientsOfInterest, on=['idcentro','idana'], how='right')[ap.columns]
diag_ex1 = pd.merge(diag, patientsOfInterest, on=['idcentro','idana'], how='right')[diag.columns]
elp_ex1 = pd.merge(elp, patientsOfInterest, on=['idcentro','idana'], how='right')[elp.columns]
ei_ex1 = pd.merge(ei, patientsOfInterest, on=['idcentro','idana'], how='right')[ei.columns]
pdf_ex1 = pd.merge(pdf, patientsOfInterest, on=['idcentro','idana'], how='right')[pdf.columns]
pdnf_ex1 = pd.merge(pdnf, patientsOfInterest, on=['idcentro','idana'], how='right')[pdnf.columns]
pnd_ex1 = pd.merge(pnd, patientsOfInterest, on=['idcentro','idana'], how='right')[pnd.columns]

## Exercise 4

**Modify the actual ranges of** ***esamilaboratorioparameteri*** **-** see this Table:

| Code | Descriptive Name | True Range |
|----------|----------|----------|
| AMD004 | Systolic blood pressure | 40 <= x <= 200 |
| AMD005 | Diastolic blood pressure | 40 <= x <= 130 |
| AMD007 | Fasting blood glucose | 50 <= x <= 500 |
| AMD008 | HbAlc | 5 <= x <= 15 |
| AMD009 | Creatininemia | Not Available |
| AMD111 | Microalbuminuria | Not Available |
| STITCH001 | BMI | Not Available |
| STITCH002 | LDL Choresterlo | 30 <= x <= 30|
| STITCH003 | Non-HDL Cholesterlo | 60 <= x <= 330 |
| STITCH004 | eGFR MDRD | Not Available |
| STITCH005 | eGFR CKD-EPI | Not Available |

What we are doing is to use a **MinMaxScaler** from *sklearn* library so as to make the values described in the upper table fit into the true ranges. For the **NAN** ranges, we do nothing.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

NAN = np.nan
newRanges = [['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005'],
            [40.0,40.0,50.0,5.0,NAN,NAN,NAN,30.0,60.0,NAN,NAN],
            [200.0,130.0,500.0,15.0,NAN,NAN,NAN,300.0,330.0,NAN,NAN]]

def scaling(df, label, minValue, maxValue) -> pd.DataFrame:
    if (minValue != NAN) and (maxValue != NAN) and (df[df.codiceamd == label].shape[0]>0):
        aux = df[df.codiceamd == label].copy()
        min_max_scaler = MinMaxScaler(feature_range=(minValue,maxValue))
        aux[['valore']] = min_max_scaler.fit_transform(aux[['valore']])
        df[df.codiceamd == label] = aux
    return df


# Let's apply the function to a new DataFrame copied from elp_new
elp_ex4 = elp_ex1.copy()
for k in range(len(newRanges[0])):
    elp_ex4 = scaling(elp_ex4, newRanges[0][k], newRanges[1][k], newRanges[2][k])

Now we ensure manually that it has worked by printing the maximum and minimum values for each AMD Code

In [ ]:
labels = ['AMD004', 'AMD005', 'AMD007', 'AMD008', 'AMD009', 'AMD111', 'STITCH001', 'STITCH002', 'STITCH003', 'STITCH004', 'STITCH005']

print("BEFORE RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp_ex1[elp_ex1.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")

print("\n\nAFTER RE-SCALING:\n")
print("    AMD    Min  /  Max")
print("========================")

for label in labels:
    aux = elp_ex4[elp_ex4.codiceamd == label].filter(['valore']).values.tolist()
    if len(aux) > 0:
        print(f"{label}:    {np.min(aux):.1f} / {np.max(aux):.1f}")
    else:
        print(f"{label}:  is null")


So it matches with the table given by the exercise

## Exercise 5

**Cohort selection and label definition** - use only those patients that after all the previous steps contain at least two events before calculating the label. Let $\mathcal{P}=\lbrace p_1, \cdots,p_n\rbrace$ be the set of all patients in the dataset. Let $d(e^i_k)$ be the date of the last event $e_k$ for patient $p_i\in\mathcal{P}$. The label of the patient $p_i$ is calculated as follows:

$y(p_i)=\begin{dcases}
                1 & \text{if, within }d(e^i_k)-6 \text{ months, }p_i\text{ has a cardiovascular event}\\
                0 & \text{otherwise}
        \end{dcases}$

Eliminate the patients that have a trajectory shorten or equal to 6 months.


We first create a DataFrame that contains all the patients and their cardiovascular events, each one with its date (it's already computed as **diag_cardEvents**):

In [ ]:
diag_cardEvents.head()

Now for each patient of the frame **diag_cardEvents**, we are going to take its two last cardiovascular events dates, so as to check if they happened with less than 6 months of difference. For this, we define the function **labeling**

In [ ]:
def labeling(df) -> pd.DataFrame:

    # First we define a dataframe that contains the ids and the dates of every cardiovascular event, sorted by descending date
    cardEvents_sorted = df.sort_values('data',ascending=False)

    # Now we create a new dataset called classes, that represents the result of the function y(p_i) defined previously:
    classes = pd.DataFrame(columns=['idcentro','idana','class'])

    # Now for each patient (idcentro,idana), we check if it verify the condition explained previously:
    for label, subDataframe in cardEvents_sorted.groupby(['idcentro','idana']):

        dates = subDataframe['data'].dropna().values

        # We ignore those patients who have just one event
        if len(dates) > 1:

            # We extract the first and the 2 lastest dates and convert them into datetime objects
            minDate = dates[-1]
            maxDates = dates[0:2]
            minDate = dt.date.fromisoformat(minDate)
            maxDates = dt.date.fromisoformat(maxDates[0]), dt.date.fromisoformat(maxDates[1])

            # We ignore patients with a trajectory shorter than 6 months (6*30 days)
            if maxDates[0]-minDate > dt.timedelta(days=6*30):

                # We calculate the difference between the latest 2 dates and compare it with 6 months
                c = maxDates[0]-maxDates[1] < dt.timedelta(days=6*30)

                # We add a new column with the label and the class of the patient
                classes.loc[classes.count()[0]] = [label[0], label[1], int(c)]

    return classes




Now, in the **classes** dataframe, we have labeled the patients that have more than one event and a trajectory larger than 6 months

In [ ]:
labels = labeling(diag_cardEvents)

In [ ]:
print(f"We still have: {labels.shape[0]}/{patientsOfInterest.shape[0]} patients")

So we do an inner merge operation again with every dataframe to take just the patients of the frame **classes**

In [ ]:
ap_ex5 = pd.merge(ap_ex2, labels, on=['idcentro','idana'], how='right')[ap_ex2.columns]
diag_ex5 = pd.merge(diag_ex2, labels, on=['idcentro','idana'], how='right')[diag_ex2.columns]
elp_ex5 = pd.merge(elp_ex4, labels, on=['idcentro','idana'], how='right')[elp_ex4.columns]
ei_ex5 = pd.merge(ei_ex2, labels, on=['idcentro','idana'], how='right')[ei_ex2.columns]
pdf_ex5 = pd.merge(pdf_ex2, labels, on=['idcentro','idana'], how='right')[pdf_ex2.columns]
pdnf_ex5 = pd.merge(pdnf_ex2, labels, on=['idcentro','idana'], how='right')[pdnf_ex2.columns]
pnd_ex5 = pd.merge(pnd_ex2, labels, on=['idcentro','idana'], how='right')[pnd_ex2.columns]
patientsOfInterest_labeled = pd.merge(patientsOfInterest, labels, on=['idcentro','idana'], how='right')
